## *De novo* TSS finding by CROWN-Seq and ReCappable-Seq

Jianheng Liu (Fox) @ Jaffrey Lab, May 31st, 2023

Concat: jil4026@med.cornell.edu

**Usage:** Modify `Prepartion` section to fit your computer. Modify `Variable` section before each run.All outputs will be saved in the `workpath`. Logs will be saved in the notebook. Don't close the `Notebook` before it finished (keep the backend at least)!

**Note 1:** In Jupyter notebook, `!` means using `bash` in the cell, and `$` means using python variables.

**Note 2:** This panel notebook should be compatible with `Python 3`. But please use `Python 2` to run the analytic scripts.

**Note 3:** I used Gencode v34 in the article for compatiblity. Here I use Ensembl release 104 for illustration. Newer annotations are normally better.

**Note 4:** This notebook is for paired-end reads. If you are using single-end, switch to the scripts end with `_SE.py`. And don't forget to change the parameters.

In [1]:
# local time
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
print("Started:", current_time)

# Show notebook directory
!pwd -P

Started: 05/31/23 16:12:50
/home/fox/Projects/CROWN-Seq_example


### 0. Variable here

In [2]:
name = "A549_WT" # file name prefix
folder = "A549_run/"
path = "./" # output path: workpath=path/folder
workpath = path + "/" + folder + "/"
read1 = "A549_rep1_example.R1.fastq.gz" # fastq or fastq.gz
read2 = "A549_rep1_example.R2.fastq.gz"

### 1. Preparation

In [3]:
ref_genome_index = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/"
ref_genome = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa"
tx_db_exons = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.tx.bed"
tx_db_tss = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.anno.tss"
faidx = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.fai"
annot = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.anno"

**Software** (For python, use the fixed version)

In [4]:
python = '/home/fox/Software/bin/python'
bowtie2 = '/home/fox/Software/bin/python2'
samtools = '/home/fox/Software/samtools/1.16/bin/samtools'
cutadapt = '/home/fox/Software/bin/cutadapt'
hisat2_path = '/home/fox/Software/hisat2/2.1.0/'
umitools = '/home/fox/Software/bin/umi_tools'
bedtools = '/home/fox/Software/bedtools/2.29.1/bedtools'

**Scripts**

In [5]:
hisat2_script = "../A2G_hisat2.py"
bam2bed_script  = "../Align_to_cap_starts_PE.py"
annot_script_1 = "../collaspe_bed_annotations_v2.py"
annot_script_2 = "../collaspe_bed_annotations_fix_other_exons.py"
bed_annotate_base = "../genome_flanking_v3_bed_annot.py"
bed_annotate_tpm = "../get_tpm.py"
tss_filters = "../ReCappabble-seq_caller_ben_version_v2.py"

**Create the workpath if not exist, then move to the workpath**

In [6]:
import os
if os.path.isdir(workpath) == False:
    os.mkdir(workpath)
os.chdir(workpath)

### 2. QC

In [7]:
adapter_1 = "AGATCGGAAGAGCACACGTC"
adapter_2 = "ATATNNNNNNNNAGATCGGAAGAGCGTCGTG"  # UMI = NNNNNNNNATAT

cutadapt_out_r1 = "read1.cutadapt.fastq"
cutadapt_out_r2 = "read2.cutadapt.fastq"

!$cutadapt -m 32 -j 4 -q 20 -e 0.25 -a $adapter_1 -A $adapter_2 -o $cutadapt_out_r1 -p $cutadapt_out_r2 ../$read1 ../$read2

This is cutadapt 4.1 with Python 3.9.14
Command line parameters: -m 32 -j 4 -q 20 -e 0.25 -a AGATCGGAAGAGCACACGTC -A ATATNNNNNNNNAGATCGGAAGAGCGTCGTG -o read1.cutadapt.fastq -p read2.cutadapt.fastq ../A549_rep1_example.R1.fastq.gz ../A549_rep1_example.R2.fastq.gz
Processing paired-end reads on 4 cores ...
Done           00:00:08     1,000,000 reads @   8.6 µs/read;   6.99 M reads/minute
Finished in 8.58 s (9 µs/read; 6.99 M reads/minute).

=== Summary ===

Total read pairs processed:          1,000,000
  Read 1 with adapter:                 959,182 (95.9%)
  Read 2 with adapter:                 355,235 (35.5%)

== Read fate breakdown ==
Pairs that were too short:             726,001 (72.6%)
Pairs written (passing filters):       273,999 (27.4%)

Total basepairs processed:   202,000,000 bp
  Read 1:   101,000,000 bp
  Read 2:   101,000,000 bp
Quality-trimmed:               1,311,002 bp (0.6%)
  Read 1:       728,357 bp
  Read 2:       582,645 bp
Total written (filtered):     36,210,167 b

In [8]:
# Extract UMI
UMI_out_r1 = "read1.UMI.fastq"
UMI_out_r2 = "read2.UMI.fastq"

# A problem about UMI-tools...

umitools_cmd = "{umitools} extract -p NNNNNNNNNNNN -I {cutadapt_out_r1} -S {UMI_out_r1} --read2-in {cutadapt_out_r2} --read2-out {UMI_out_r2}".format(umitools=umitools, cutadapt_out_r1=cutadapt_out_r1, cutadapt_out_r2=cutadapt_out_r2, UMI_out_r1=UMI_out_r1, UMI_out_r2=UMI_out_r2)

import os
os.system(umitools_cmd)


# UMI-tools version: 1.1.1
# output generated by extract -p NNNNNNNNNNNN -I read1.cutadapt.fastq -S read1.UMI.fastq --read2-in read2.cutadapt.fastq --read2-out read2.UMI.fastq
# job started at Wed May 31 16:13:00 2023 on orion -- 000c4b15-28a5-40f5-972b-297908658b64
# pid: 1660982, system: Linux 5.15.0-69-generic #76~20.04.1-Ubuntu SMP Mon Mar 20 15:54:19 UTC 2023 x86_64
# blacklist                               : None
# compresslevel                           : 6
# correct_umi_threshold                   : 0
# either_read                             : False
# either_read_resolve                     : discard
# error_correct_cell                      : False
# extract_method                          : string
# filter_cell_barcode                     : None
# filter_cell_barcodes                    : False
# filter_umi                              : None
# filtered_out                            : None
# filtered_out2                           : None
# ignore_suffix                     

0

### 2. Mapping

In [9]:
!$python $hisat2_script -F $UMI_out_r1 -R $UMI_out_r2 -o hisat2_genome -I $ref_genome_index --index-prefix HISAT2 --hisat2-path $hisat2_path --del-convert --del-sam

[2023-05-31 16:13:08]Converting read1.UMI.fastq, A2G...
[2023-05-31 16:13:09]Converting read2.UMI.fastq, T2C...
[2023-05-31 16:13:10]Mapping with hisat2, TEMP prefix: hisat2_1661061
[2023-05-31 16:13:27]A2G report:
[2023-05-31 16:13:28]T2C report:
[2023-05-31 16:13:28]Handling SAM outputs...
[2023-05-31 16:13:50]Completed successfully:
 Total reads: 273999
 Unique mapping: 158348 (57.791%)
   A2G: 80851 (29.51%)
   T2C: 77497 (28.28%)
 Multiple mapping: 32421 (11.833%)
 Unmapped: 83230 (30.376%)


In [10]:
!$samtools sort -@ 4 -m 4G -o hisat2_genome.sorted.bam hisat2_genome.bam
!$samtools index hisat2_genome.sorted.bam

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


In [11]:
!$umitools dedup --stdin=hisat2_genome.sorted.bam --log=umi.logs --method=unique > hisat2_genome.sorted.umi.bam

In [12]:
!samtools sort -@ 4 -m 4G -o hisat2_genome.sorted.umi.sorted.bam  hisat2_genome.sorted.umi.bam
!samtools index hisat2_genome.sorted.umi.sorted.bam 

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


### 3. Fetch 5' ends of the reads

In [13]:
bed_output = "{}.bed".format(name)
sorted_bed = "{}.sorted.bed".format(name)

!$python $bam2bed_script hisat2_genome.sorted.umi.sorted.bam > $bed_output
!$bedtools sort -i $bed_output > $sorted_bed

### 4. Find the closest annotations for the 5' ends

In [14]:
bed_tss_annot = "{}.closest.tss.bed".format(name)
bed_tx_annot = "{}.closest.tx.bed".format(name)

!$bedtools closest -s -D b -a $sorted_bed -b $tx_db_tss > $bed_tss_annot
!$bedtools closest -s -D b -a $sorted_bed -b $tx_db_exons > $bed_tx_annot

### 5. Find the best annotation for the 5' ends

First, annotate by the closest TSS, ambiguous annoations are in the format "*{name of the gene}_{distance}"

In [15]:
bed_best_tss = "{}.byTSS.bed".format(name)

!$python $annot_script_1 -i $bed_tss_annot -a $annot  -o $bed_best_tss

Second, the ambiguous annotations will be re-annotated by the transcripts. If the site is still ambiguous, the will be annotated by "*{name of the gene}_{distance}" again.

In [16]:
bed_best_tss_tx = "{}.byTSS.byTx.bed".format(name)

!$python $annot_script_2 -i $bed_best_tss -I $bed_tx_annot -a $annot -o $bed_best_tss_tx

### 6. Simple annotations for the base and TPM

In [17]:
bed_with_base = "{}.byTSS.byTx.base.bed".format(name)
bed_with_tpm = "{}.byTSS.byTx.base.tpm.bed".format(name)

!$python $bed_annotate_base -f $ref_genome -F 0 -i $bed_best_tss_tx > $bed_with_base
!$python $bed_annotate_tpm $bed_with_base > $bed_with_tpm

### 7. Last step, find out which one is high-confident

In [18]:
output_prefix = "{}.called".format(name)

!$python $tss_filters -i $bed_with_tpm -o $output_prefix

  0%|                                          | 5/8215 [00:00<07:14, 18.89it/s]/home/fox/Projects/CROWN-Seq_example/A549_run/../ReCappabble-seq_caller_ben_version_v2.py:71: RuntimeWarning: invalid value encountered in divide
  zscores = (tpms - np.mean(tpms))/np.std(tpms)
100%|█████████████████████████████████████| 8215/8215 [00:01<00:00, 7304.53it/s]


Finally, there will be two outputs:

The high-confident sites will be stored in:

`output_prefix`.ben.passed.bed 

While the sites do not pass the filters will be

`output_prefix`.ben.filtered_out.bed

Both of these two files are important for further analysis.

### 8. When and where am I

In [19]:
!pwd -P
!ls -l

/home/fox/Projects/CROWN-Seq_example/A549_run
total 334420
-rw-rw-r-- 1 fox fox   999050 May 31 16:14 A549_WT.bed
-rw-rw-r-- 1 fox fox  2224094 May 31 16:14 A549_WT.byTSS.bed
-rw-rw-r-- 1 fox fox  2082458 May 31 16:15 A549_WT.byTSS.byTx.base.bed
-rw-rw-r-- 1 fox fox  2753453 May 31 16:15 A549_WT.byTSS.byTx.base.tpm.bed
-rw-rw-r-- 1 fox fox  2008158 May 31 16:15 A549_WT.byTSS.byTx.bed
-rw-rw-r-- 1 fox fox  3657061 May 31 16:15 A549_WT.called.ben.filtered_out.bed
-rw-rw-r-- 1 fox fox   937726 May 31 16:15 A549_WT.called.ben.passed.bed
-rw-rw-r-- 1 fox fox  3366972 May 31 16:14 A549_WT.closest.tss.bed
-rw-rw-r-- 1 fox fox 13057734 May 31 16:14 A549_WT.closest.tx.bed
-rw-rw-r-- 1 fox fox   999050 May 31 16:14 A549_WT.sorted.bed
-rw-rw-r-- 1 fox fox 13808418 May 31 16:13 hisat2_genome.bam
-rw-rw-r-- 1 fox fox  7987967 May 31 16:13 hisat2_genome.multimappers.bam
-rw-rw-r-- 1 fox fox 10352027 May 31 16:13 hisat2_genome.sorted.bam
-rw-rw-r-- 1 fox fox  1723552 May 31 16:13 hisat2_genome.sorted

In [20]:
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
print("Finished:", current_time)

Finished: 05/31/23 16:15:44


## Don't forget to save the Notebook.